In [ ]:
import os
from os.path import join
from io import StringIO
from textwrap import dedent

import sympy
from sympy import Symbol, sqrt, cos, pi, symbols
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from collections import defaultdict

import qnet
from qnet.algebra import *

import QDYN

from src.notebook_plots_v1 import plot_bs_decay, display_hamiltonian, display_eq, show_summary_dicke
from src.single_sided_network_v1 import network_slh
from src.dicke_single_model_v1 import write_dicke_single_model, err_dicke_single

from doit.tools import register_doit_as_IPython_magic
import clusterjob

In [ ]:
qnet.init_printing()

In [ ]:
DOIT_CONFIG = {
    'backend': 'json',
    'dep_file': '.doit_db/2017-06-07_mg_dicke1_QSL.json',
}
register_doit_as_IPython_magic()

In [ ]:
clusterjob.JobScript.read_defaults('./config/copper.ini')

$
\newcommand{ket}[1]{\vert #1 \rangle}
\newcommand{bra}[1]{\langle #1 \vert}
\newcommand{Op}[1]{\hat{#1}}
$

# QSL for Single-Excitation Dicke state

## action wrappers

*This notebook used a buggy version of QDYN where the MCWF trajectory optimization would use a simple non-Hermtian propagation for the forward propagation*

Data is now located in `./data/dicke1_QSL_xxx/`

In [ ]:
def update_config(rf, lambda_a, iter_stop):
    config = QDYN.config.read_config_file(join(rf, 'config'))
    config['oct']['iter_stop'] = iter_stop
    for pulse_config in config['pulse']:
        pulse_config['oct_lambda_a'] = lambda_a
    QDYN.config.write_config(config, join(rf, 'config'))

In [ ]:
def submit_optimization(rf, n_trajs, task):
    """Asynchronously run optimization"""
    if os.path.isfile(join(rf, 'oct.job.dump')):
        return
    body = dedent(r'''
    {module_load}

    cd $CLUSTERJOB_WORKDIR
    pwd
	export MPICH_NO_BUFFER_ALIAS_CHECK=1
    OMP_NUM_THREADS=1 {runner} qdyn_optimize --n-trajs={n_trajs} \
         --J_T=J_T_sm .
    ''')
    taskname = "oct_%s" % task.name.split(":")[-1]
    jobscript = clusterjob.JobScript(
        body=body, jobname=taskname, workdir=rf,
        stdout='oct.log')
    runner = {
        'copper': 'aprun -B',
        'mlhpc': 'mpirun -n %d' % n_trajs}
    if jobscript.cluster_name != 'copper':
        jobscript.ppn = int(n_trajs)
        if n_trajs == 1:
            runner['mlhpc'] = ''
    jobscript.runner = runner[jobscript.cluster_name]
    jobscript.n_trajs = str(int(n_trajs))
    run = jobscript.submit(cache_id=taskname)
    run.dump(join(rf, 'oct.job.dump'))

In [ ]:
def wait_for_clusterjob(dumpfile):
    """Wait until the clusterjob.AsyncResult cached in the given dumpfile ends"""
    try:
        run = clusterjob.AsyncResult.load(dumpfile)
        run.wait()
        os.unlink(dumpfile)
        return run.successful()
    except OSError:
        pass

## custom uptodate routines

In [ ]:
from src.qdyn_model_v1 import pulses_uptodate

## task definitions

In [ ]:
def runfolder(row):
    int_part, frac_part = ("%.2f" % row['T']).split('.')
    root = './data/dicke1_QSL_xxx/'
    if frac_part == '00':
        rf = 'rf_%dnodes_T%s' % (row['n_nodes'], int_part)
    else:
        rf = 'rf_%dnodes_T%s_%s' % (row['n_nodes'], int_part, frac_part)
    return join(root, rf)

In [ ]:
def task_create_runfolder():
    """Create all necessary runfolders for the runs defined in params_df"""
    jobs = {}
    slh_models = {}
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        n_nodes = int(row['n_nodes'])
        try:
            slh = slh_models[n_nodes]
        except KeyError:
            slh = network_slh(
                n_cavity=2, n_nodes=n_nodes, topology='driven_bs_fb')
            slh_models[n_nodes] = slh
        if n_nodes not in slh_models:
            slh_mode
        if rf in jobs:
            continue
        jobs[rf] = {
            'name': str(rf),
            'actions': [
                (write_dicke_single_model, [slh, ], dict(
                    rf=rf, T=row['T'], theta=0, nt=500,
                    kappa=1.0, E0_cycles=2, mcwf=True, non_herm=True,
                    lambda_a=row['lambda_a'],
                    iter_stop=int(row['iter_stop'])))],
            'targets': [join(rf, 'config')],
            'uptodate': [True, ] # up to date if target exists
        }
    for job in jobs.values():
        yield job

In [ ]:
def task_update_runfolder():
    """For every row in params_df, update the config file in the appropriate
    runfolder with the value in that row"""
    rf_jobs = defaultdict(list)
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        # we only update the config after any earlier optimization has finished
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        rf_jobs[rf].append(ind)
        yield {
            'name': str(ind),
            'actions': [
                (update_config, [], dict(
                    rf=rf, lambda_a=row['lambda_a'],
                    iter_stop=int(row['iter_stop'])))],
            'file_dep': [join(rf, 'config')],
            'uptodate': [False, ],  # always run task
            'task_dep': task_dep}


In [ ]:
def task_submit_optimization():
    """Run optimization for every runfolder from params_df"""
    rf_jobs = defaultdict(list)
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        task_dep.append('update_runfolder:%s' % ind)
        yield {
            'name': str(ind),
            'actions': [
                (submit_optimization, [rf, ], dict(n_trajs=row['n_trajs']))],
                # 'task' keyword arg is added automatically
            'task_dep': task_dep,
            'uptodate': [(pulses_uptodate, [], {'rf': rf}), ],
        }

In [ ]:
def task_wait_for_optimization():
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        yield {
            'name': str(ind),
            'task_dep': ['submit_optimization:%d' % ind],
            'actions': [
                (wait_for_clusterjob, [join(rf, 'oct.job.dump')], {}),]}

##  OCT Submission

In [ ]:
params_data_str = r'''
# n_nodes   T     lambda_a  n_trajs   iter_stop
        2   1.00     0.001       32        1000
        2   1.00    0.0005       32       10000
        2   1.00    0.0005       32       15000
        2   2.00     0.001       32        1000
        2   2.00    0.0005       32       10000
        2   2.00    0.0005       32       15000
        2   3.00     0.001       32        1000
        2   3.00    0.0005       32       10000
        2   3.00    0.0005       32       15000
        2   4.00     0.001       32        1000
        2   4.00    0.0005       32       10000
        2   5.00     0.001       32        1000
        2   7.00     0.001       32        1000
        2  10.00     0.001       32        1000
        2  10.00    0.0005       32        2000
        3   1.00     0.001       32        1000
        3   1.00    0.0005       32       10000
        3   2.00     0.001       32        1000
        3   2.00    0.0005       32       10000
        3   3.00     0.001       32        1000
        3   3.00    0.0005       32       10000
        3   4.00     0.001       32        1000
        3   4.00    0.0005       32        4000
        3   5.00     0.001       32        1000
        3   5.00    0.0005       32        4000
        3   8.00     0.001       32        1000
        3   8.00    0.0005       32        2000
        3   9.00     0.001       32        1000
        3   9.00    0.0005       32        2000
        3  10.00     0.001       32        1000
        3  10.00    0.0005       32        2000
        4   0.50     0.001       32        1000
        4   1.00     0.001       32        1000
        4   1.00    0.0005       32       10000
        4   1.00    0.0005       32       15000
        4   2.00     0.001       32        1000
        4   2.00    0.0005       32       10000
        4   2.00    0.0005       32       15000
        4   3.00     0.001       32        1000
        4   3.00    0.0005       32       10000
        4   3.00    0.0005       32       15000
        4   4.00     0.001       32        1000
        4   4.00    0.0005       32       10000
        4   5.00     0.001       32        1000
        4   5.00    0.0005       32       10000
        4   6.00     0.001       32        1000
        4   6.00    0.0005       32       10000
        4   7.00     0.001       32        1000
        4   7.00    0.0005       32       10000
        4   8.00     0.001       32        1000
        4   8.00    0.0005       32        4000
        4   9.00     0.001       32        1000
        4   9.00    0.0005       32        4000
        4  10.00     0.001       32        1000
        4  10.00    0.0005       32        4000
        5   1.00     0.001       32        1000
        5   1.00    0.0005       32       10000
        5   1.00    0.0005       32       15000
        5   2.00     0.001       32        1000
        5   2.00    0.0005       32       10000
        5   2.00    0.0005       32       15000
        5   3.00     0.001       32        1000
        5   3.00    0.0005       32       10000
        5   3.00    0.0005       32       15000
        5   4.00     0.001       32        1000
        5   4.00    0.0005       32       10000
        5   4.00    0.0005       32       15000
        5   5.00     0.001       32        1000
        5   5.00    0.0005       32       10000
        5   5.00    0.0005       32       15000
        5   6.00     0.001       32        1000
        5   6.00    0.0005       32       10000
        5   7.00     0.001       32        1000
        5   7.00    0.0005       32       10000
        5   8.00     0.001       32        1000
        5   8.00    0.0005       32       10000
        5   9.00     0.001       32        1000
        5   9.00    0.0005       32       10000
        5   9.00    0.0005       32       20000
        5  10.00     0.001       32        1000
        5  10.00    0.0005       32        8000
        6   1.00     0.001       32        1000
        6   1.00    0.0005       32       10000
        6   2.00     0.001       32        1000
        6   2.00    0.0005       32       10000
        6   3.00     0.001       32        1000
        6   3.00    0.0005       32       10000
        6   4.00     0.001       32        1000
        6   4.00    0.0005       32       10000
        6   5.00     0.001       32        1000
        6   5.00    0.0005       32       10000
        6   5.00    0.0005       32       15000
        6   6.00    0.0005       32       10000
        6   6.00    0.0005       32       15000
        6   7.00    0.0005       32       10000
        6   7.00    0.0005       32       15000
        6   8.00     0.001       32        1000
        6   8.00    0.0005       32       10000
        6   8.00    0.0005       32       15000
        6   9.00     0.001       32        1000
        6   9.00    0.0005       32       10000
        6   9.00    0.0005       32       15000
        6  10.00     0.001       32        1000
        6  10.00    0.0005       32       10000
        7   5.00     0.001       32        1000
        7   5.00    0.0005       32       10000
        7   8.00     0.001       32        1000
        7   8.00    0.0005       32       10000
        7   9.00     0.001       32        1000
        7   9.00    0.0005       32       10000
        7  10.00     0.001       32        1000
        7  10.00    0.0005       32       10000
'''
params_df = pd.read_fwf(
        StringIO(params_data_str), comment='#', header=1,
        names=['n_nodes', 'T', 'lambda_a', 'n_trajs', 'iter_stop'],
        converters={'n_nodes': int, 'T': float, 'lambda_a': float,
                    'n_trajs': int, 'iter_stop': int})

In [ ]:
print(params_df.to_string())

In [ ]:
import logging
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(level=logging.DEBUG, filename='./data/dicke1_QSL_clusterjob.log')

In [ ]:
%doit -n 20 wait_for_optimization